In [ ]:
# 이미지 전처리

bbox = []
for label in labels:
    width = label["W"]
    height = label["H"]
    point_str = label["Point(x,y)"]

    x, y = point_str.split(",")
    x = float(x)
    y = float(y)
    width = float(width)
    height = float(height)

    # x_min = int(x * owidth)
    # y_min = int(y * oheight)
    # x_max = int((x + owidth) * owidth)
    # y_max = int((y + oheight) * owidth)

bbox.append((x, y, width, height))

print(x)
print(y)
print(width)
print(height)

for i, bbox_info in enumerate(bbox):
    x1, y1, x2, y2 = bbox_info
    x1, y1, x2, y2 = (
        (x1 * owidth) - (x2/2) * owidth,
        (y1 * oheight) - (y2/2) * oheight,
        (x1 * owidth) + (x2/2) * owidth,
        (y1 * oheight) + (y2/2) * oheight,
        )
    print(x1, y1, x2, y2)

    # 바운딩 박스 그리기
    plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color="red")

    # 라벨 텍스트 추가
    plt.text(x1, y1, f"Image {i + 1}", color="white")

# MNet V2

In [ ]:
from torch import nn
from torch import Tensor
from typing import Callable, Any, Optional, List


__all__ = ['MobileNetV2', 'mobilenet_v2']


model_urls = {
    'mobilenet_v2': 'https://download.pytorch.org/models/mobilenet_v2-b0353104.pth',
}


def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class ConvBNReLU(nn.Sequential):
    def __init__(
        self,
        in_planes: int, # 입력 특징 맵의 채널 수
        out_planes: int, # 출력 특징 맵의 채널 수
        kernel_size: int = 3, # 필터 크기
        stride: int = 1, # 필터의 이동 거리
        groups: int = 1, # 연산에 사용되는 그룹 수
        norm_layer: Optional[Callable[..., nn.Module]] = None # 정규화 레이어 클래스 기본값 nn.BatchNorm2d
    ) -> None:
        padding = (kernel_size - 1) // 2 # 입력에 추가되는 패딩의 양, 커널 크기가 홀수일 때 중앙 정렬

        # 레이어 쌓기
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        super(ConvBNReLU, self).__init__(
            # 입력 특징 맵을 출력 특징 맵으로 변환하는 컨볼루션레이어 생성
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),

            # 컨볼루션 결과에 적용되는 정규화 레이어 생성 기본값 BatchNorm2d, 출력 채널 수에 맞게 채널별 정규화 진행
            norm_layer(out_planes),

            # 활성화 함수 ReLU6(0보다 작으면 0, 6보다 크면 6), inplace=True는 입력을 직접 수정
            nn.ReLU6(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(
        self,
        inp: int, # 입력 채널 수
        oup: int, # 출력 채널 수
        stride: int, # 필터 이동 거리
        expand_ratio: int, # 확장 비율, 입력 채널을 얼마나 확장할지
        norm_layer: Optional[Callable[..., nn.Module]] = None # 정규화 기본 레이어 batchnorm2d
    ) -> None:
        super(InvertedResidual, self).__init__()
        self.stride = stride
        # stride는 반드시 1 또는 2이어야 하므로 조건을 걸어 둡니다.
        assert stride in [1, 2]

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        # expansion factor를 이용하여 channel을 확장합니다.
        hidden_dim = int(round(inp * expand_ratio))
        # stride가 1인 경우에만 residual block을 사용합니다.
        # skip connection을 사용하는 경우 input과 output의 크기가 같아야 합니다.
        self.use_res_connect = (self.stride == 1) and (inp == oup)

        # Inverted Residual 연산
        # 레이어 쌓기
        layers: List[nn.Module] = []
        if expand_ratio != 1:
            # point-wise convolution
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1, norm_layer=norm_layer))

        layers.extend([
            # depth-wise convolution
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=norm_layer),
            # point-wise linear convolution
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        # use_res_connect인 경우만 connection을 연결합니다.
        # use_res_connect : stride가 1이고 input과 output의 채널 수가 같은 경우 True
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(
        self,
        num_classes: int = 1000,
        width_mult: float = 1.0,
        inverted_residual_setting: Optional[List[List[int]]] = None,
        round_nearest: int = 8,
        block: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        """
        MobileNet V2 main class
        Args:
            num_classes (int): Number of classes
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
            block: Module specifying inverted residual building block for mobilenet
            norm_layer: Module specifying the normalization layer to use
        """
        super(MobileNetV2, self).__init__()

        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        input_channel = 32
        last_channel = 1280

        # t : expansion factor
        # c : output channel의 수
        # n : 반복 횟수
        # s : stride
        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 2],
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # only check the first element, assuming user knows t,c,n,s are required
        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        # building first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features: List[nn.Module] = [ConvBNReLU(3, input_channel, stride=2, norm_layer=norm_layer)]


        # Inverted Residual Block을 생성합니다.
        # features에 feature들의 정보를 차례대로 저장합니다.
        for t, c, n, s in inverted_residual_setting:
            # width multiplier는 layer의 채널 수를 일정 비율로 줄이는 역할을 합니다.
            output_channel = _make_divisible(c * width_mult, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t, norm_layer=norm_layer))
                input_channel = output_channel

        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1, norm_layer=norm_layer))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # This exists since TorchScript doesn't support inheritance, so the superclass method
        # (this one) needs to have a name other than `forward` that can be accessed in a subclass
        x = self.features(x)
        # Cannot use "squeeze" as batch-size can be 1 => must use reshape with x.shape[0]
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1)).reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def mobilenet_v2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> MobileNetV2:
    """
    Constructs a MobileNetV2 architecture from
    `"MobileNetV2: Inverted Residuals and Linear Bottlenecks" <https://arxiv.org/abs/1801.04381>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = MobileNetV2(**kwargs)
    if pretrained:
        try:
            from torch.hub import load_state_dict_from_url
        except ImportError:
            from torch.utils.model_zoo import load_url as load_state_dict_from_url
        state_dict = load_state_dict_from_url(
            'https://download.pytorch.org/models/mobilenet_v2-b0353104.pth', progress=True)
        model.load_state_dict(state_dict)
    return model

if __name__ == '__main__':
    net = mobilenet_v2(True)

In [ ]:
# 다른 버전

import torch
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import os

# 일반 3x3 convolution
def conv_3x3(in_channel, out_channel, stride):
    return nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU6(inplace=True)
    )

# 일반 1x1 convolution
def conv_1x1(in_channel, out_channel):
    return nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(out_channel),
        nn.ReLU6(inplace=True)
    )

# pointwise용 1x1 convolution
def pointwise_conv(in_channel, out_channel):
    return nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(out_channel)
    )

# depthwise용 3x3 convolution
def depthwise_conv(in_channel, stride):
    return nn.Sequential(
        nn.Conv2d(in_channel, in_channel, kernel_size=3, stride=stride, groups=in_channel, padding=1),
        nn.BatchNorm2d(in_channel),
        nn.ReLU6(inplace=True)
    )

# 기본적으로 사용되는 역 Residual Block
class InvertedResidual(nn.Module):
    def __init__(self, in_channel, out_channel, stride, expand_ratio):
        super(InvertedResidual,self).__init__()
        self.stride = stride

        # expand_ratio = Inverted Residual block 안에서 채널을 얼마나 확장시키는지에 대한 값, 본 논문에서는 t=6으로 하여 확장시키고 있음
        # inverted residual 내부 확장된 채널값을 가지게 하기 위함
        hidden_dim = int(in_channel * expand_ratio)

        # skip connection이 가능한지 조건을 줌
        self.use_res_connect = self.stride == 1 and in_channel == out_channel

        layers = []

        # inverted residual 내부에서 확장시켜야 할 때
        if expand_ratio != 1:
            # 확장시키는 1x1 convolution 사용
            layers.append(conv_1x1(in_channel,hidden_dim))
        # 확장시키던지 안시키던지 그 후에는 depthwise와 pointwise를 진행하여 함, 이는 MobileNetV1에서 나온 이론에 근거
        layers.extend([
            # depth wise convolution
            depthwise_conv(hidden_dim, stride=stride),
            # point wise convolution
            pointwise_conv(hidden_dim, out_channel)
        ])

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        ## skip connection 조건이 맞으면 해줌
        if self.use_res_connect:
            return x + self.layers(x)
        else :
            return self.layers(x)


class MobileNetV2(nn.Module):
    def __init__(self,img_channel, n_class):
        super(MobileNetV2,self).__init__()
        # 첫 번째 c 값
        input_channel = 32
        # 마지막 c 값
        last_channel = 1280

        # 본 논문에서 inverted residual block에 할당하는 값
        # t : 확장 비율, 기본으로 설정된 channel 값에 해당 숫자를 곱해서 사용
        # c : out_channel 값
        # n : 반복 횟수
        # s : 첫 번째 반복시 stride
        residual_setting = [
            #t, c,  n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1]
        ]

        # 첫 번째 layer
        self.first_conv = conv_3x3(img_channel, input_channel, stride = 2)

        # bottleneck 구조의 Inverted Residual block layers
        layers = []
        for t, c, n, s in residual_setting:
            # 반복의 첫 번째 s만 지정된 stride값으로 입력 그 외에는 1로 바꿈 그 이유는 Downsampling은 한 번만 해야하므로..
            for i in range(n):
                if i == 0:
                    stride = s
                else :
                    stride = 1
                layers.append(InvertedResidual(input_channel, c, stride = stride, expand_ratio= t))
                input_channel = c

        self.layers = nn.Sequential(*layers)

        # 마지막 layer
        self.last_conv = conv_1x1(input_channel, last_channel) #input = 320 / output = 1280
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(last_channel,n_class)

    def forward(self, x):
        x = self.first_conv(x)
        x = self.layers(x)
        x = self.last_conv(x)
        x = self.avg_pool(x).view(-1, 1280)
        x = self.classifier(x)
        return x

# 모델 확인
if __name__=="__main__":
    model = MobileNetV2(img_channel=3, n_class=1000)
    summary(model,(3,224,224), device = "cpu")

In [ ]:
# cuda 사용 가능 시 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# 변수설정
batch_size = 64
epochs = 10
lr = 0.001
data_dir = '/content/bread-1'
# /content/bread-1



# transform 설정
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])




model = MobileNetV2(img_channel=3, n_class=10).to(device)




# 비용함수, 옵티마이저 정의

# "Cross-Entropy"는 두 확률 분포 간의 차이를 측정하는 방법 중 하나
# 딥 러닝에서는 일반적으로 모델의 출력을 확률로 해석하고,
# 실제 클래스 레이블과 예측 확률 사이의 차이를 최소화하는 것이 목표
# 모델 출력과 실제 레이블 간의 차이 계산: 모델의 출력은 클래스에 대한 확률 분포로 해석됩니다.
# Cross-Entropy 손실은 모델의 출력과 실제 레이블 간의 차이를 계산합니다.
# 손실 값 계산: 계산된 차이를 바탕으로 손실 값을 계산합니다. 이 손실 값은 모델이 얼마나 잘 예측하고 있는지를 나타냅니다.
# 역전파 수행: 손실 값을 사용하여 역전파를 수행하여 모델의 가중치를 업데이트합니다.
# 이를 통해 모델이 더 나은 예측을 할 수 있도록 학습됩니다.
# 매개변수
# weight (optional): 각 클래스에 대한 손실 가중치를 지정할 수 있는 매개변수입니다.
# 이를 사용하여 특정 클래스에 대한 오류에 더 큰 패널티를 부여하거나 무시할 수 있습니다.
# ignore_index (optional): 무시해야 할 클래스 레이블의 인덱스를 지정할 수 있는 매개변수입니다.
# 이를 사용하여 손실을 계산할 때 특정 클래스를 무시할 수 있습니다.

# 모델의 가중치를 업데이트하는 데 사용되는 옵티마이저
# 확률적 경사 하강법(SGD)의 변형 중 하나로, 모멘텀(momentum) 및 이동 평균(rolling average) 기법을 사용하여 학습 속도를 조절하는 방법
# Adaptive Learning Rate: 각 파라미터마다 동적으로 학습률을 조정합니다.
# 이는 각 파라미터에 대해 학습률을 자동으로 조절하여 최적의 학습 속도를 찾을 수 있도록 합니다.
# Momentum: Adam은 모멘텀을 사용하여 이전 그래디언트의 지수 가중 이동 평균을 유지합니다.
# 이를 통해 기울기의 방향을 예측하고, 빠른 수렴을 도와줍니다.
# Bias-Correction: Adam은 편향 보정(bias-correction)을 수행하여 초기에 그래디언트 추정치가 편향되는 것을 보정합니다.
# 매개변수
# params: 최적화해야 하는 파라미터들의 리스트입니다. 주로 모델의 parameters() 메서드를 사용하여 이를 전달합니다.
# lr (학습률): 학습률은 각 파라미터의 가중치 업데이트에 적용되는 스케일링 요소입니다.
# 적절한 학습률은 학습 과정의 안정성과 수렴 속도에 영향을 미칩니다.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
best_acc = 0

In [ ]:
# pytorch에서 가져오기
from torchvision import models
mobilenet_v2 = models.mobilenet_v2()
# summary(mobilenet_v2, (3,224,224), device="cpu")

배치 사이즈 (Batch Size): 배치 사이즈는 한 번에 모델에 공급되는 데이터의 샘플
  수를 나타냅니다. 예를 들어, 배치 사이즈가 32인 경우, 모델은 32개의 데이터 포인트로 한 번에 학습됩니다.  

에포크 (Epoch): 에포크는 전체 데이터셋이 모델에 대해 한 번 학습되는 것을   나타냅니다. 따라서 1 에포크는 전체 데이터셋에 대한 단일 학습 주기를 의미합니다.   에포크가 끝나면 모델은 새로운 에포크에 대한 학습을 시작합니다.  

배치 사이즈와 에포크는 모두 모델의 학습 과정에 영향을 미칩니다. 더 큰 배치 사이즈는 메모리 사용량을 늘리지만 학습 속도를 높일 수 있습니다. 반면 더 작은 배치 사이즈는 메모리를 적게 사용하지만 학습 속도가 느릴 수 있습니다. 에포크 수는 모델이 데이터셋을 얼마나 많이 학습하는지를 결정하며, 학습 과정의 안정성과 정확도에 영향을 줄 수 있습니다.

기존 cnn 연산량 많고 메모리 사용량 많음.  
-> 모바일에 적합하지 않아 모바일에 적합한 모델 구현  

MobileNetV1  
: Depthwise Separable Convolution (depthwise conv + pointwise conv)  
: 세로 방향 채널 3개를 depthwise conv 과 pointwise conv 으로 나누어서 실행  
: depthwise conv 은 채널을 하나씩 떼어서 채널 하나에 대해 각각 다른 필터로 컨볼루션 한 다음에 붙이는 작업  
: pointwise conv = one by one conv 의 다른 이름  
=> 실행하면 스탠다드 3X3 컨볼루션과 똑같이 되나 스탠다드 컨볼루션은 3D 에서 하는데 가로 세로 방향의 2D와 채널방향의 1D로 연산하는데  
mobilenet은 2개로 나누어서
depthwise로 가로 세로 방향으로만 연산하고  
pointwise로 채널 방향만 고려해서 연산  
즉 분리시켜 연산량 등을 많이 줄인 모델
연산량 비교하면 약 8~9배 정도 줄임  

3x3 과 1x1을 합쳐서 하나의 컨볼루션으로 만듦  
3X3 depthwise conv  
        ↓  
        BN  
        ↓  
      ReLu (exception 논문은 사이에 쓰지 않는게 좋다고 하긴 함)  
        ↓  
    1X1 conv  
        ↓  
        BN  
        ↓  
      ReLu        


with multiplier = 채널수를 기본을 1이라 했을 때 3/4, 2/4, 1/4(0.75, 0.5, 0.25) 로 줄이는 것  

resolution multiplier = 들어가는 이미지의 레졸루션을 일정한 비율로 줄이는 것  

# 모바일넷 V2
depthwise separable convolutions + Linear Bottlenecks + Inverted Residuals

## Linear Bottlenecks
: 이미지을 받았을 때 레이어들이 manifold of interest(데이터가 특정한 방식으로 조직되어 있는 형태나 구조를 나타냄) 형성한다고 사람들은 생각함, 저차원 임베딩(: 특정한 데이터를 고차원의 벡터 공간으로 매핑, 이미지를 임베딩하여 고차원의 이미지 데이터를 저차원의 벡터로 표현),
1. ReLU 통과 후 선형을 따른다. (음수는 0 양수는 냅두는 형태)
=> 로우 디멘젼으로 채널 수 줄이는 거 할 때 Linear bottleneck layer를 쓰면 더 효과적으로 할 수 있음 (ReLU 안 쓰고 ReLU를 쓴게 결국 Linear transform 한거처럼 동작하니), bottleneck 쓸 때 ReLU 쓰면 성능 떨어짐

## Residual Blocks
처음에 들어가는 것은 채널이 많은 구조에서 one by one 컨볼루션 bottleneck으로 채널 수를 줄인 다음에 필요한 3X3하고 Residual connection이 끝에 와서 다시 더해져야 하니까 이거랑 디멘젼이랑 맞추기 위해 one by one 컨볼루션을 다시 채널을 늘리는 작업을 구성함.
wide -> narrow(bottleneck) -> wide approach


## Inverted Residuals
 이 논문에서는 반대로 한다.
one by one 컨볼루션이 relu없이 linear bottleneck이 오퍼레이션 할 때 큰 채널 수에서 작은 거로 갈 때 큰 채널에 있는 정보를 잃지 않고 다 담고 있다고 가정하기때문에 정보를 다 담고 있는 거에서 Residual connection을 만들어도 문제가 없다고 생각하는 것이다. 이렇게 하면 메모리 사용량에 있어서도 효율적이다.
정 반대로 처음에 작은 narrow -> wide -> narrow approach 로 된다. 앞에서 쓴거와 정 반대로 작용하기에 inverted residuals라고 이름을 붙인다.

# 합치면 Bottelneck Residual Blocks : 기본 빌딩 블록
: 내로우 상태로 입력이 들어오면 1X1 expansion layer를 써서 채널 키우고 -> batch nomalization -> ReLU6(활성화 함수, 6이상 되는 애는 6으로)  -> 3X3 depthwise Conv -> batch -> ReLU6 -> 1X1 Projection Layer (=Linear bottleneck: 채널수 다시 줄임) -> batch -> residual connection 만나서 덧셈함
: 중간에 채널을 얼만큼 팽창시킬건지 expansion factor t = 6 으로만 진행 (5~10 정도가 성능이 가장 좋았다고 논문에서 말함)
: 식만 보면 v1 보다 연산량이 더 많아 보이지만 실제로 보면 채널수가 v2가 더 적은 채널을 가지고 할 수 있어 실제로는 연산량이 60% 정도로 수준이다.
: 네트워크 expressiveness 는 expansion layer에서 얼마나 채널 수를 늘려주냐에 따라 결정되고
capacity는 projection layer에서 얼마나 줄여주냐에 따라 관련이 있다. 안에서 분리가 되어 있으니 따로 따로 컨트롤 할 수 있으니 좋다라고 설명

### tip. stride
: 합성곱 신경망(Convolutional Neural Network, CNN)에서 필터(또는 커널)가 입력 데이터를 순회하는 간격을 나타냄. 스트라이드는 필터가 입력 데이터를 얼마나 이동할지를 결정하며, 필터가 한 번에 이동하는 거리를 나타냄
: 스트라이드를 사용하면 출력 특성 맵의 크기를 조절할 수 있습니다. 보통 스트라이드가 1인 경우, 필터는 한 칸씩 이동하여 입력 데이터를 순회합니다. 이 경우 출력 특성 맵의 크기는 입력 특성 맵의 크기와 동일합니다. 하지만 스트라이드가 2인 경우, 필터는 두 칸씩 이동하여 입력 데이터를 순회하므로 출력 특성 맵의 크기는 입력 특성 맵의 크기의 반으로 줄어듭니다.
:  스트라이드가 커질수록 공간적인 정보의 손실이 발생할 수 있으므로, 적절한 스트라이드 값을 선택하는 것이 중요

### tip. resolution = 해상도

## Hyper Parameters
- input resolution ( 96 ~ 224 )
- width multiplier ( 0.35 ~ 1.4 ) - 채널 수 늘렸다 줄였다.

: 메모리 효율적으로 쓸 수 있다. bottleneck block 안에서 메모리를 언제 어디서 왔다 갔다 해야하는지 잘 생각하면 처음과 마지막에서 사용하면 내부에서는 내부에서 처리할 수 있다고 생각하니
메모리 오퍼레이션이 처음과 마지막에서 일어난다고 했을 때 처음이 작으면 작을수록 좋으니 내로우한 채널을 가지고 있으면 더 유리하다는 얘기.
: 여기서 더 메모리를 효율적으로 쓸 수 있게 하는 방법도 설명
채널을 n조각 내서 몇개 채널을 먼저 계산해서 보내는 식으로 하면 메모리를 좀 더 효과적으로 쓸 수 있으나 너무 잘게 자르면 성능이 떨어지니 2~5정도로 하면 굉장히 효율적으로 할 수 있다고 말하고 있음

# stride = 1 VS stride = 2 비교

1일때만 residual connection이 있다.
2 일때는 왜 없냐?
추측하면 가로 세로가 줄어드니 레지듀얼 커넥션이 날라갈 때 (레지듀얼 커넥션은 마지막에 더해진다.) 이미 expansion하면서 채널 수를 만들었기 때문에 그리고 1X1 layer에서 또 한번 조정할 거기 때문에 그거에 맞게 residual connection도 2배로 늘리는거 하기 싫으니 그냥 안 쓴거 같다.
이렇게 해도 성능에 크게 문제 없는거로 추청됨

## SSDLite
: 바운딩 박스에 대한거를 중간중간 피쳐맵에서 뽑아서 가져가는데 3X3 컨볼루션 연산을 하는데 이 연산을 다 depthwise sepaable 연산으로 바꾼것

: 객체 검출에서
=> 연산량 굉장히 많이 줄인다. 파람수도 많이 줄임
==> MNet V2 + SSDLite 하면 성능이 굉장히 좋다.


: 시맨틱 분할(semantic segmentation: 이미지의 각 픽셀을 해당하는 클래스 레이블로 분류하는 작업을 의미, 이미지에서 객체의 경계를 식별하는 것이 아니라, 각 픽셀을 해당하는 클래스에 할당하여 이미지를 구분하는 것)에서
=> MNet V2 + deeplabv3 : resnet 보다는 성능이 약간 낮지만 파람수나 연산량이 비교가 안될 정도로 좋다.

: 임베딩 시스템에서 구현한거라 모바일 쪽에서 하는 게 좋다.
레즈넷은 마이크로 소프트 리서치에서 만든거라 환경이 다르다

# MNet V1 구현 코드 1

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#  2D 컨볼루션, 필터수, 필터크기, 제로패팅, relu 활성화 함수
# 배치 정규화 레이어 모델에 추가
def add_block(model, num_filters):
  model.add(layers.Conv2D(num_filters, (1,1), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.DepthwiseConv2D((3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())

In [ ]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# 데이터 크기 변환
x_train=x_train.reshape(-1,28,28,1)
x_test= x_test.reshape(-1,28,28,1)
x_train, x_test = x_train / 255.0, x_test /255.0

In [ ]:
# 빈 모델 생성
# Sequential 모델은 레이어를 선형으로 쌓아 구성하는 가장 간단한 신경망 구조
# add() 메서드를 사용하여 각 레이어를 순차적으로 추가한 후,
# compile() 메서드를 사용하여 모델을 컴파일하고 학습을 시작할 준비
model=models.Sequential()

In [ ]:
# 784개의 입력을 받는 Dense 레이어 하나와 10개의 출력을 가지는 Dense 레이어 하나로 이루어진 Sequential 모델을 생성
# Conv2D 레이어를 추가
# 32= 필터(또는 커널)의 수(감지하고자 하는 특징의 수 결정)
# (3,3) = 필터의 크기
# padding='same': 컨볼루션 연산, 제로 패딩, 입력 이미지 크기 = 출력 이미지 크기
# 활성화 함수 relu 사용 (음수 입력에 대해 0으로 출력하고 양수 입력은 그대로 출력하는 비선형 함수)
# ㄴ check 필요 relu6 사용한다고 한거 같은데??
# input_shape=(28,28,1) : 28X28 크기의 그레이스케일 이미지 입력(1차원)
# => 28X28 크기의 그레이스케일 이미지를 32개의 3X3 필터로 컨볼루션 후 relu 활성화 함수 적용하는 Conv2D 레이어 구축
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28,28,1)))

In [ ]:
# 배치 정규화 레이어 모델에 추가
# 배치 정규화는 신경망의 안정성과 학습 속도를 향상시키기 위해 사용되는 기술 중 하나
model.add(layers.BatchNormalization())

In [ ]:
# filters에 있는 값들을 순회하면서 모델에 add_block 함수를 사용하여 블록을 추가하는 것
# 각 값은 모델에 추가할 컨볼루션 레이어의 필터 수
filters=[64,128,256,512,1024]

#  주어진 개수의 필터를 갖는 컨볼루션 블록을 모델에 추가하는 함수
for num_filters in filters:
  add_block(model, num_filters)

In [ ]:
# Flatten 레이어를 모델에 추가하는 함수 : 다차원 배열을 1차원으로 펼쳐주는 역할, 완전 연결 레이어를 사용할 때 사용
# 이미지 처리에서는 일반적으로 Convolutional Neural Network(CNN)을 통해 특징을 추출하고,
# 이후에 완전 연결 레이어를 사용하여 분류를 수행
# CNN의 출력은 다차원 배열로 나타나는데, 이를 Flatten 레이어를 통해 1차원으로 변환하여 완전 연결 레이어의 입력으로 사용
model.add(layers.Flatten())

In [ ]:
# Dense 레이어는 fully connected 레이어로, 이전 레이어의 모든 뉴런과 연결되어 있는 레이어
# 10: 레이어의 출력 뉴런의 수, 출력 클래스의 개수롸 동일해야 함
# 10개의 출력을 가진 fully connected 레이어 생성
# 10가지의 클래스 중 하나를 예측하는 다중 클래스 분류 작업에 사용

# 활성화 함수 softmax : 다중 클래스 분류 문제에서 출력 확률을 계산하는 데 사용
# 출력 벡터의 각 요소는 해당 클래스에 대한 확률
# 출력을 확률 분포로 변환하여 모든 클래스에 대한 확률의 합이 1이 되도록 함
# => 10개의 출력을 가진 fully connected 레이어를 모델에 추가하고
#    이 레이어의 출력에 softmax 활성화 함수를 적용하여 다중 클래스 분류를 수행할 준비 완료
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# 테스트 10시 7분 진행

In [ ]:
# 모델 컴파일
# => adam 옵티마이저로 모델 컴파일, 손실함수 sparse_categorical_crossentropy 사용, 성능지표로 정확도(Accuracy) 사용

# optimizer='adam' : Adam은 경사 하강법의 한 종류로, 모델의 가중치를 업데이트하는데 사용되는 최적화 알고리즘 중 하나
#                   Adam은 다양한 하이퍼파라미터를 자동으로 조정하여 최적의 학습률을 찾아줌

# loss='sparse_categorical_crossentropy': 손실 함수(loss function) sparse categorical crossentropy
# 다중 클래스 분류 문제에서 사용되는 손실 함수
# 정수 형태의 클래스 레이블을 입력으로 받아 확률 분포와 비교하여 손실을 계산
# 출력 레이어의 활성화 함수로 softmax를 사용하는 경우에 주로 사용

# metrics=['accuracy']: 모델의 성능을 평가할 지표(metric)로 정확도(accuracy)를 선택

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 8233s 4s/step - loss: 0.3483 - accuracy: 0.9649 - val_loss: 1.2331 - val_accuracy: 0.9236
Epoch 2/10
 840/1875 [============>.................] - ETA: 1:10:23 - loss: 0.1971 - accuracy: 0.9794

KeyboardInterrupt: 

In [ ]:
# 모델 평가
# verbose=0 : 출력을 얼마나 자세하게 표시할지 제어,
#             0: 출력을 표시하지 않음을 의미
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print('Test Accuracy:', test_acc)